In [ ]:
import wandb
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from util import populate_plt_settings, get_column_width, get_fig_size, get_latex_float

In [ ]:
api = wandb.Api()
all_runs = api.runs('kennychufk/alluvion')

In [ ]:
selected_runs = []
for run in all_runs:
    if (len(run.tags) and run.tags[0]=='pellet_vis_sweep0' and run.state=='finished'):
        selected_runs.append(run)

In [ ]:
kinematic_vis = []
vis = []
bvis = []
best_losses = []
for run in selected_runs:
    vis.append(run.summary['vis_real'])
    bvis.append(run.summary['bvis_real'])
#     vis.append(run.summary['best_x'][0])
#     bvis.append(run.summary['best_x'][1])
    kinematic_vis.append(run.config['kinematic_viscosity_real'])
    best_losses.append(run.summary['best_loss'])

In [ ]:
plt.plot(kinematic_vis, vis, marker='x')

In [ ]:
plt.plot(kinematic_vis, bvis, marker='x')

In [ ]:
populate_plt_settings(plt)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=get_fig_size(get_column_width()))

scale = 1e6
vis_color = 'tab:blue'
ax.scatter(np.array(kinematic_vis)*scale, np.array(vis)*scale, marker='.', color=vis_color)
ax.set_ylabel(r"Parameter value (cSt)")

bvis_color = 'tab:red'
ax.scatter(np.array(kinematic_vis)*scale, np.array(bvis)*scale, marker='.', color=bvis_color)

ax.set_xlabel(r"Kinematic viscosity $\nu$ (cSt)")

reg = LinearRegression().fit(np.array(kinematic_vis).reshape(-1, 1), vis)
breg = LinearRegression().fit(np.array(kinematic_vis).reshape(-1, 1), bvis)
xlim_upper_scaled = 21
xlim_upper = xlim_upper_scaled * 1e-6
vis_y0 = reg.predict(np.array([0]).reshape(-1, 1))[0]
bvis_y0 = breg.predict(np.array([0]).reshape(-1, 1))[0]
vis_y1 = reg.predict(np.array([xlim_upper]).reshape(-1, 1))[0]
bvis_y1 = breg.predict(np.array([xlim_upper]).reshape(-1, 1))[0]
ax.plot([0, xlim_upper_scaled], np.array([vis_y0, vis_y1])*scale, color=vis_color, label=r"SPH viscosity $\nu_{\mathcal{F}}$")
ax.plot([0, xlim_upper_scaled], np.array([bvis_y0, bvis_y1])*scale, color=bvis_color, label=r"SPH boundary viscosity $\nu_{\mathcal{B}}$")

vis_coef_str = f"{reg.coef_[0]:.3f}"
vis_intercept_scaled = reg.intercept_*scale
vis_intercept_str = f"{'+' if vis_intercept_scaled>0 else ''}{get_latex_float(vis_intercept_scaled)}"
ax.annotate(
    r"$"+"\\nu_{\mathcal{F}}="+vis_coef_str+"\\nu "+vis_intercept_str+" \\mathrm{cSt}"+r"$",
    xy=(8, 6), xycoords='data',
    bbox=dict(boxstyle="round,pad=0.2", facecolor="white", edgecolor=vis_color))

bvis_coef_str = f"{breg.coef_[0]:.3f}"
bvis_intercept_scaled = breg.intercept_*scale
bvis_intercept_str = f"{'+' if bvis_intercept_scaled>0 else ''}{get_latex_float(bvis_intercept_scaled)}"
ax.annotate(
    r"$"+"\\nu_{\mathcal{B}}="+bvis_coef_str+"\\nu "+bvis_intercept_str+" \\mathrm{cSt}"+r"$",
    xy=(0.5, 53), xycoords='data',
    bbox=dict(boxstyle="round,pad=0.2", facecolor="white", edgecolor=bvis_color))


ax.set_xlim(0,xlim_upper_scaled)
ax.set_ylim(0,90)
ax.legend(facecolor='white', frameon=True, edgecolor='None')
fig.tight_layout(pad=0.05) # pad is 1.08 by default https://stackoverflow.com/a/59252633
fig.savefig('vary-vis.pgf')

In [ ]:
plt.plot(kinematic_vis, best_losses)

In [ ]:
reg.coef_[0], reg.intercept_

In [ ]:
breg.coef_[0], breg.intercept_

In [ ]:
reg.predict(np.array([[1e-6]])), breg.predict(np.array([[1e-6]]))